In [2]:
import os
from timeit import default_timer as timer
from datetime import datetime, timedelta
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import entropy

In [3]:
path_to_data='/scratch/spf248/covid/data'
start_date='01/01/2020'
end_date='04/16/2020'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
day=list(range(8,20))
night=list(range(8))+list(range(20,24))

# Compute entropy of neighborhood visits

In [ ]:
print('Import')
start = timer()
data_dirs=glob(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
pings_geocoded = pd.concat(
    pd.read_parquet(parquet_file)
    for data_dir in data_dirs for parquet_file in Path(data_dir).glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Process')
start = timer()
pings_geocoded['date']=pd.to_datetime(pings_geocoded.time.dt.date)
pings_geocoded['hour']=pings_geocoded.time.dt.hour
print("Done in", round(timer()-start), "sec")  

In [ ]:
def get_entropy(data):
    """Calculates entropy of the passed `pd.Series`
    """
    p_data = data.value_counts() # counts occurrence of each value
    return entropy(p_data)       # get entropy from counts

In [ ]:
print('Compute entropy of neighborhoods')
start = timer()
entropy_date_id=pings_geocoded.groupby(['date','cuebiq_id']).agg(entropy=(admin_id,get_entropy),n_pings=('time','count'))
entropy_daytime_id=pings_geocoded[pings_geocoded.hour.isin(day)].groupby(['date','cuebiq_id']).agg(entropy=(admin_id,get_entropy),n_pings=('time','count'))
entropy_nighttime_id=pings_geocoded[pings_geocoded.hour.isin(night)].groupby(['date','cuebiq_id']).agg(entropy=(admin_id,get_entropy),n_pings=('time','count'))
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Process')
start = timer()
entropy_date_id=entropy_date_id[entropy_date_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
entropy_daytime_id=entropy_daytime_id[entropy_daytime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
entropy_nighttime_id=entropy_nighttime_id[entropy_nighttime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
print("Done in", round(timer()-start), "sec")

In [6]:
print('Save')
start = timer()
entropy_date_id.to_csv(os.path.join(path_to_data,source,'processed',country,'entropy_date_id.csv'))
entropy_daytime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'entropy_daytime_id.csv'))
entropy_nighttime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'entropy_nighttime_id.csv'))
print("Done in", round(timer()-start), "sec")

Save
Done in 61 sec


# Figures